In [ ]:
using JuMP,HiGHS,LaTeXStrings, GLPK, StatsPlots, Plots, DelimitedFiles

# Input Data

In [ ]:
#Input data

asset_quantity = readdlm("data/sample_asset_quantity.csv", ',', Float64)
asset_tiers = readdlm("data/sample_asset_tiers.csv", ',', Float64)
asset_value = readdlm("data/sample_asset_value.csv", ',', Float64)

account_exposure = readdlm("data/sample_account_exposure.csv", ',', Float64)
account_durations = readdlm("data/sample_account_duration.csv", ',', Float64)


#haircuts_with_ineligibility = readdlm("sample_data/", ',', Float64)/100
haircuts = readdlm("data/sample_haircuts.csv", ',', Float64)/100


#single_limits = readdlm("data/sample_single_limits.csv", ',', Float64)

In [ ]:
# Auxillary variables

tier_set = [0.2, 0.5, 0.8]

n_assets = length(asset_tiers)

n_accounts = length(account_durations)



cost_factor_matrix = Array{Float64}(undef, length(asset_tiers), length(account_durations))

for i in 1:n_assets
    for j in 1:n_accounts
        cost_factor_matrix[i,j] = abs(asset_tiers[i] - account_durations[j])
    end
end

cost_factor_matrix = round.(cost_factor_matrix, digits = 2);

print(cost_factor_matrix)




# Constructing JuMP model

In [ ]:

model = Model()

#Q 
@variable(model, Q[i = 1:n_assets, j = 1:n_accounts], lower_bound = 0, upper_bound = 1)


#Consistency constraint
@constraint(model,[i = 1:n_assets],sum(Q[i,j] for j in 1:n_accounts) <= 1)

#Requirement constraint
@constraint(model, [j = 1:n_accounts],
    sum(Q[i,j]*asset_quantity[i]*asset_value[i]*haircuts[i,j] for i in 1:n_assets )
    >= account_exposure[j])


#Single Limits
# @constraint(model, [i = 1:n_assets, j =1:n_accounts], 
#     Q[i,j]*asset_quantity[i] <= single_limits[i,j])

#Objective_function
@objective(model, Min, sum(Q[i,j]*cost_factor_matrix[i,j] for i in 1:n_assets,j in 1:n_accounts));
print(model)

# Solving with HiGHs


In [ ]:
set_optimizer(model, HiGHS.Optimizer)
set_optimizer_attribute(model, "solver", "simplex")
optimize!(model)

In [ ]:

println("Objective value is $(objective_value(model))")


# Export to .csv

In [ ]:
Q_values_HiGHS_simplex = zeros((n_assets, n_accounts))


for i in 1:n_assets
    for j in 1:n_accounts
        Q_values_HiGHS_simplex[i,j] = value(Q[i,j])
        if value(Q[i,j]) != 0 
            println("Q_{$(i), $(j)} = $(value(Q[i,j]))")
        end
    end
end

println(Q_values_HiGHS_simplex)
#store all Values to an array


# Uncomment line below to save to csv files

#writedlm("Q_values_HiGHS_simplex.csv", Q_values_HiGHS_simplex, ',')